<a href="https://colab.research.google.com/github/MarkoMilenovic01/Sentiment-Analysis---DL-Models-review/blob/main/Sentiment_Analysis_Fine_Tuned_Transformer_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🤖 Experiment Summary: Fine-Tuning DistilBERT for Sentiment Analysis

### Dataset
- **Source**: IMDB Movie Reviews (50,000 labeled reviews)
- **Split**: 80% training / 20% testing
- **Task**: Binary classification (positive / negative sentiment)

---

### 🧹 Preprocessing (Enhanced)
- **Contractions expanded** (e.g. “don’t” → “do not”)
- **Emoticons mapped** (e.g. `:)` → “smile”)
- **Removed**: HTML tags, URLs, digits
- **Negation tagging**: (e.g. "not good" → "not_good")
- **Stopword removal** *(except: "no", "not", "never", "hardly")*
- **Lemmatization** using WordNet

---

### 🔄 Tokenization
- Model: `distilbert-base-uncased`
- Tokenizer: `DistilBertTokenizerFast`
- Max length: 128 tokens
- Format: torch tensors for Trainer API

---

### ⚙️ Model & Training Setup
- **Model**: `DistilBertForSequenceClassification` (2 output labels)
- **Epochs**: 2
- **Batch size**: 16
- **Learning rate**: 2e-5
- **Weight decay**: 0.01
- **Trainer**: Hugging Face `Trainer` with accuracy and F1 metrics

---

### 📊 Results

| Metric           | Value      |
|------------------|------------|
| **Test Accuracy**| ✅ **90.04%** |
| **Macro F1 Score**| 📏 **0.8950** |
| **ROC AUC**      | ⭐ **0.9641**  |

**ROC AUC Interpretation**: The model is very confident in distinguishing between positive and negative reviews.

---

### 🧠 Observations

- DistilBERT performs **extremely well** with minimal fine-tuning and semantic preprocessing.
- Compared to classical models (TF-IDF + Linear SVM @ 91.6%), it has slightly lower accuracy, but **significantly higher ROC AUC**, indicating better probability calibration.
- Deep learning models like DistilBERT benefit more from **probability-based evaluation** (e.g., ROC AUC) than raw accuracy alone.

---

### ✅ Conclusion

Fine-tuned DistilBERT reached **90.04% accuracy**, **0.8950 macro F1**, and **0.9641 ROC AUC**, proving itself a highly capable model for sentiment classification.

However, it slightly underperformed compared to classical models like TF-IDF + Linear SVM. This is likely due to several factors:

- The **training set size (40,000 examples)**, while moderate, is **relatively small** for fully fine-tuning a deep transformer model like DistilBERT.
- The IMDB dataset is **clean, balanced, and rich in sentiment cues**, making it well-suited for simpler models like Logistic Regression and SVM.
- Classical models with TF-IDF features can capture most sentiment-bearing terms (e.g., "not good", "absolutely loved") directly and effectively.
- **No hyperparameter tuning** or **learning rate scheduling** was applied during fine-tuning, and training was limited to **2 epochs**, which may have prevented DistilBERT from reaching its full potential.

Despite this, DistilBERT produced the **best-calibrated predictions**, making it a strong candidate for deployment scenarios where **confidence scores and ranking predictions** are critical.


In [ ]:
!pip install -q --upgrade transformers datasets evaluate


In [ ]:
!pip install nltk contractions
import nltk; nltk.download('wordnet'); nltk.download('omw-1.4'); nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import re
import contractions
import numpy as np
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from datasets import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import evaluate

lemmatizer = WordNetLemmatizer()
STOP = set(stopwords.words('english')) - {"no", "not", "never", "hardly"}

EMOTICON_MAP = {
    ":)": " smile ",
    ":-)": " smile ",
    ":(": " sad ",
    ":-(": " sad ",
    ";)": " wink ",
}

def clean_text_enhanced(text: str) -> str:
    text = contractions.fix(text)

    for emo, tok in EMOTICON_MAP.items():
        text = text.replace(emo, tok)

    text = re.sub(r"<.*?>|http\S+|www\.\S+", " ", text)

    text = re.sub(r"\b\d+\b", "[NUM]", text)

    text = re.sub(r"[^A-Za-z0-9\s!?.,]", " ", text)

    text = re.sub(r"([!?.,])\1+", r"\1", text)

    words = text.split()
    for i, word in enumerate(words):
        if word.lower() in ["no", "not", "never", "hardly"] and i + 1 < len(words):
            words[i+1] = word.lower() + "_" + words[i+1]
    text = " ".join(words)

    tokens = text.lower().split()
    clean_tokens = []
    for tok in tokens:
        if tok not in STOP:
            lemma = lemmatizer.lemmatize(tok)
            clean_tokens.append(lemma)

    return " ".join(clean_tokens)





In [ ]:
df = pd.read_csv("IMDB Dataset.csv")
df["text"]  = df["review"].apply(clean_text_enhanced)
df["label"] = (df["sentiment"] == "positive").astype(int)

In [ ]:
ds = Dataset.from_pandas(df[["text","label"]])
splits   = ds.train_test_split(test_size=0.2, seed=42)
train_ds = splits["train"]
eval_ds  = splits["test"]

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
def tokenize_fn(example):
    return tokenizer(
      example["text"],
      truncation=True,
      padding="max_length",
      max_length=128
    )
train_ds = train_ds.map(tokenize_fn, batched=True)
eval_ds  = eval_ds.map(tokenize_fn,  batched=True)

train_ds.set_format(type="torch", columns=["input_ids","attention_mask","label"])
eval_ds .set_format(type="torch", columns=["input_ids","attention_mask","label"])

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
)
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=100,
)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    acc = accuracy_score(labels, preds)
    f1_macro = f1_score(labels, preds, average="macro")
    return {
        "accuracy": acc,
        "macro_f1": f1_macro
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

trainer.train()
results = trainer.evaluate()
acc_bert = results['eval_accuracy']*100
print(f"\n✅ DistilBERT Test Accuracy: {acc_bert:.2f}%")
print(f"📏 DistilBERT Macro F1: {results['eval_macro_f1']:.4f}")


<ipython-input-14-9864047ab4fa>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,0.173700
200,0.162400
300,0.160100
400,0.143100
500,0.166300
600,0.121400
700,0.197200
800,0.144600
900,0.158800
1000,0.149900



✅ DistilBERT Test Accuracy: 89.50%
📏 DistilBERT Macro F1: 0.8950


In [ ]:
from sklearn.metrics import roc_auc_score

bert_preds = trainer.predict(eval_ds)

import numpy as np
probs = np.exp(bert_preds.predictions) / np.exp(bert_preds.predictions).sum(axis=1, keepdims=True)
y_prob_bert = probs[:, 1]

y_true_bert = np.array(eval_ds["label"])

auc_bert = roc_auc_score(y_true_bert, y_prob_bert)
print(f"DistilBERT ROC AUC: {auc_bert:.4f}")


DistilBERT ROC AUC: 0.9641
